In [5]:
#대형주 10개 종목 모의투자
import requests
import json
import keyring
import pandas as pd
import time
import numpy as np
import datetime
from datetime import timedelta
import schedule

#종합계좌
keyring.set_password('real_app_key', '박인성', 'PSMovGbOkW9YiMVTmtpJhSuPVmxtQciSLcgQ')
keyring.set_password('real_app_secret', '박인성', 'E54hYNDgbs+yFY2doIWpyV/5Rhy7hwutBZlvlS+8UW3uG2/s1mpEKn4pNNFImMsUeuK/ya7P3QPHKGk+pP5WFrH9cYcX5N7F2Vg4qhsgg1efupe3m3TzoedmzJNiwMyYVOPe92T92KNJRZ6mC3UOJuKyFCaSytQCrPFjnk2713UKuezEwMg=')

#모의계좌
keyring.set_password('mock_app_key', '박인성', 'PSqlCpPd6AhBCSbEkV47ZjZj5GWFRRHlE34s')
keyring.set_password('mock_app_secret', '박인성', 'E54hYNDgbs+yFY2doIWpyV/5Rhy7hwutBZlvlS+8UW3uG2/s1mpEKn4pNNFImMsUeuK/ya7P3QPHKGk+pP5WFrH9cYcX5N7F2Vg4qhsgg1efupe3m3TzoedmzJNiwMyYVOPe92T92KNJRZ6mC3UOJuKyFCaSytQCrPFjnk2713UKuezEwMg=')

# API Key
app_key = keyring.get_password('mock_app_key', '박인성')
app_secret = keyring.get_password('mock_app_secret', '박인성')


#접근토큰 발급
url_base = "https://openapivts.koreainvestment.com:29443" #모의투자

headers = {"content-type":"application/json"}
path = "oauth2/tokenP"
body = {"grant_type":"client_credentials",
        "appkey":app_key, 
        "appsecret":app_secret}

url = f"{url_base}/{path}"
res = requests.post(url, headers=headers, data=json.dumps(body)) # url에 데이터 전송
access_tocken = res.json()['access_token']


#해시키 발급
def hashkey(datas):
  path = "uapi/hashkey"
  url = f"{url_base}/{path}"
  headers = {
    'content-Type' : 'application/json',
    'appKey' : app_key,
    'appSecret' : app_secret,
    }
  res = requests.post(url, headers=headers, data=json.dumps(datas))
  hashkey = res.json()["HASH"]

  return hashkey

In [41]:
#현재가 구하기
def get_price(ticker):
    path = "uapi/domestic-stock/v1/quotations/inquire-price"
    url = f"{url_base}/{path}"

    headers = {"Content-Type":"application/json", 
            "authorization": f"Bearer {access_tocken}",
            "appKey":app_key,
            "appSecret":app_secret,
            "tr_id":"FHKST01010100"}
    
    #tr_id는 거래 id에 해당하며, 원하는 작업마다 tr_id가 다름
    params = { "fid_cond_mrkt_div_code":"J", "fid_input_iscd": ticker}

    res = requests.get(url, headers=headers, params=params)
    price = res.json()['output']['stck_prpr']
    price = int(price)
    time.sleep(0.15)

    return price

#매수, 매도
def trading(ticker, tr_id):
    path = "uapi/domestic-stock/v1/trading/order-cash"
    url = f"{url_base}/{path}"

    data = {
        "CANO": "50095584", # 계좌번호 앞 8 자리
        "ACNT_PRDT_CD": "84", # 계좌번호 뒤 2자리
        "PDNO": ticker, # 종목코드
        "ORD_DVSN": "03", # 주문방법 | 시장가: 01 , 지정가 : 00 
        "ORD_QTY": "1", # 주문 수량
        "ORD_UNPR": "0", #주문 단가(시장가의 경우0)
    }

    headers = {"Content-Type":"application/json", 
            "authorization":f"Bearer {access_tocken}",
            "appKey":app_key,
            "appSecret":app_secret,
            "tr_id": tr_id,
            "custtype":"P",
            "hashkey" : hashkey(data)}

    res = requests.post(url, headers=headers, data=json.dumps(data))


#계좌 잔고 조회
#모의투자에서는 20종목, 실제계좌에서는 50종목까지 조회가 가능
def check_account():

    output1 = []
    output2 = []
    CTX_AREA_NK100 = "" #이전 조회 output값을 넣으면 다음페이지 조회 가능 -> 연속조회 가능

    while True:        
        path = "/uapi/domestic-stock/v1/trading/inquire-balance"
        url = f"{url_base}/{path}"

        headers = {"Content-Type":"application/json", 
                "authorization":f"Bearer {access_tocken}",
                "appKey":app_key,
                "appSecret":app_secret,
                "tr_id":"VTTC8434R"}
        
        params = {
            "CANO": "50095584", # 계좌번호 앞 8 자리
            "ACNT_PRDT_CD": "84", # 계좌번호 뒤 2자리
            "AFHR_FLPR_YN": "N", #시간외 당일가 여부
            "OFL_YN": "", # 공란,
            "INQR_DVSN": "01", # 조회구분
            "UNPR_DVSN": "01", # 단가구분
            "FUND_STTL_ICLD_YN": "N", # 펀드 결제분 포함 여부
            "FNCG_AMT_AUTO_RDPT_YN": "N", # 융자금액 자동상환 여부
            "PRCS_DVSN": "00", #처리구분 (00 전일매매포함)
            "CTX_AREA_FK100": "", #연속조회검색조건100
            "CTX_AREA_NK100": CTX_AREA_NK100 #연속조회키100
        }

        res = requests.post(url, headers=headers, params=params)
        output1.append(pd.DataFrame.from_records(res.json()['output1']))

        CTX_AREA_NK100 = res.json()['ctx_area_nk100'].strip()

        if CTX_AREA_NK100 == "":
            output2.append(res.json()['output2'][0])
            break
    
    if not output1[0].empty:
        res1 = pd.concat(output1)[['pdno', 'hldg_qty']].rename(columns={
                                                        'pdno': '종목코드',
                                                        'hldg_qty': '보유수량'
                                                    }).reset_index(drop=True)
    else:
        res1 = pd.DataFrame(columns=['종목코드, 보유수량'])

    res2 = output2[0]

    return [res1, res2]



In [42]:
# 모델 포트폴리오
mp = pd.DataFrame({
    '종목코드': [
        '005930', # 삼성전자
        '373220', # LG에너지솔루션
        '000660', # Sk하이닉스
        '207940', # 삼성바이오로직스
        '051910', # LG화학
        '035420', # NAVER
        '005380', # 현대차
        '006400', # 삼성SDI
        '035720', # 카카오
        '105560', # KB금융
    ]
})

# 보유 종목과 aum 불러오기
ap, account = check_account()

# 주당 투자 금액
invert_per_stock = int(account['tot_evlu_amt']) * 0.98 / len(mp)

# 매매 구성
target = pd.DataFrame()
if ap.empty:
   target = mp
   target['보유수량'] = 0
   ap = target
else:
    target = mp.merge(ap, on = '종목코드', how='outer')



target['보유수량'] = target['보유수량'].fillna(0).apply(pd.to_numeric)

#현재가 확인
target['현재가'] = target.apply(lambda x: get_price(x.종목코드), axis = 1)

#목표수량 및 투자수량 입력
target['목표수량'] = np.where(target['종목코드'].isin(mp['종목코드'].tolist()),
                          round(invert_per_stock / target['현재가']), 0)
target['투자수량'] = target['목표수량'] - target['보유수량']

In [44]:
#시간 분할
startDt1 = datetime.datetime.now() + timedelta(minutes=10)
startDt2 = datetime.datetime.now().replace(hour=9,minute=10,second=0,microsecond=0)
startDt = max(startDt1, startDt2)
endDt = datetime.datetime.now().replace(hour=12,minute=0,second=0,microsecond=0) #원래는 15시 까지인데 지금은 12시 까지로 test

# 스케줄 초기화
schedule.clear()

# 스케줄 등록
for t in range(target.shape[0]) : 

    n = target.loc[t, '투자수량']
    position = 'VTTC0802U' if n > 0  else 'VTTC0801U'
    ticker = target.loc[t, '종목코드']

    time_list = pd.date_range(startDt, endDt, periods=abs(n))
    time_list = time_list.round(freq = 's').tolist()
    time_list_sec = [s.strftime('%H:%M:%S') for s in time_list]

    for i in time_list_sec:
        #스케줄러에 등록될 함수에 인자가 들어가는 경우
        #do(함수, 인자1, 인자2,...)
        schedule.every().day.at(i).do(trading, ticker, position)
        

In [46]:
#스케줄 실행
while True:
    schedule.run_pending()
    if datetime.datetime.now()> endDt:
        print('거래가 완료되었습니다.')
        schedule.clear()
        break

거래가 완료되었습니다.
